# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [31]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
geoapify_key = "6d15b6bb7a8a4c2baff806e956d5ebfd"

In [32]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv('../output_data/cities.csv')

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,5.81,70,0,4.12,AR,1675651896
1,1,coihaique,-45.5752,-72.0662,13.97,51,0,2.57,CL,1675652167
2,2,port alfred,-33.5906,26.8910,22.32,94,79,3.45,ZA,1675652168
3,3,ploemeur,47.7333,-3.4333,4.97,89,0,4.63,FR,1675652168
4,4,jamestown,42.0970,-79.2353,5.01,87,100,4.92,US,1675651963


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [33]:
#%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    tiles = 'OSM',
    geo = True,
    frame_width= 700,
    frame_height= 500,
    size= 'Humidity',
    scale= 0.5,
    color= 'City')

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [34]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    ((city_data_df['Max Temp'] < 27.00) & (city_data_df['Max Temp'] > 21.00))
    & (city_data_df['Wind Speed"] < 4.5)
    & (city_data_df["Cloudiness"] == 0)
,:]
# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,west bay,19.3667,-81.4167,25.04,86,0,1.03,KY,1675652172
59,59,caravelas,-17.7125,-39.2481,25.49,84,0,4.49,BR,1675652091
63,63,coahuayana,18.7333,-103.6833,21.39,84,0,1.42,MX,1675652189
215,215,tucuman,-26.8241,-65.2226,23.00,88,0,1.54,AR,1675652239
247,247,acapulco,16.8634,-99.8901,26.90,74,0,2.57,MX,1675652073
255,255,rosario,-32.9468,-60.6393,24.90,68,0,2.57,AR,1675652253
339,339,saint-pierre,-21.3393,55.4781,25.82,73,0,2.57,RE,1675652279
369,369,ormara,25.2088,64.6357,21.42,57,0,2.69,PK,1675652289
412,412,garissa,-0.4536,39.6401,22.51,80,0,3.28,KE,1675652301
484,484,cartagena,10.3997,-75.5144,25.79,83,0,3.09,CO,1675652055


### Step 3: Create a new DataFrame called `hotel_df`.

In [35]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Max Temp", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Max Temp,Lat,Lng,Humidity,Hotel Name
15,west bay,KY,25.04,19.3667,-81.4167,86,
59,caravelas,BR,25.49,-17.7125,-39.2481,84,
63,coahuayana,MX,21.39,18.7333,-103.6833,84,
215,tucuman,AR,23.00,-26.8241,-65.2226,88,
247,acapulco,MX,26.90,16.8634,-99.8901,74,
255,rosario,AR,24.90,-32.9468,-60.6393,68,
339,saint-pierre,RE,25.82,-21.3393,55.4781,73,
369,ormara,PK,21.42,25.2088,64.6357,57,
412,garissa,KE,22.51,-0.4536,39.6401,80,
484,cartagena,CO,25.79,10.3997,-75.5144,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [46]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'api_key': geoapify_key
}
# Print a message to follow up the hotel search
print('Starting hotel search')
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    # Add filter and bias parameters with the current city’s latitude and longitude to the params dictionary
    params['filter'] = f'circle:{lng},{lat},{radius}'
    params['bias'] = f'proximity:{lng},{lat}'
    # Set base URL
    base_url = 'https://api.geoapify.com/v2/places'
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, 'Hotel Name'] = name_address['features'][0]['properties']['name']
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as “No hotel found”.
        hotel_df.loc[index, 'Hotel Name'] = 'No hotel found'
    # Log the search results
    print(f"{hotel_df.loc[index,'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
# Display sample data
hotel_df

Starting hotel search
west bay - nearest hotel: Bonnie's Arch
caravelas - nearest hotel: Pousada dos Navegantes
coahuayana - nearest hotel: No hotel found
tucuman - nearest hotel: Suites San Luis
acapulco - nearest hotel: Hotel del Valle
rosario - nearest hotel: Fontana Suites
saint-pierre - nearest hotel: Tropic Hotel
ormara - nearest hotel: No hotel found
garissa - nearest hotel: Hiddig Hotel
cartagena - nearest hotel: Hotel Bellamar
puerto escondido - nearest hotel: Hotel Barlovento
ballitoville - nearest hotel: The Vineyard On Ballito


,City,Country,Max Temp,Lat,Lng,Humidity,Hotel Name
15,west bay,KY,25.04,19.3667,-81.4167,86,Bonnie's Arch
59,caravelas,BR,25.49,-17.7125,-39.2481,84,Pousada dos Navegantes
63,coahuayana,MX,21.39,18.7333,-103.6833,84,No hotel found
215,tucuman,AR,23.00,-26.8241,-65.2226,88,Suites San Luis
247,acapulco,MX,26.90,16.8634,-99.8901,74,Hotel del Valle
255,rosario,AR,24.90,-32.9468,-60.6393,68,Fontana Suites
339,saint-pierre,RE,25.82,-21.3393,55.4781,73,Tropic Hotel
369,ormara,PK,21.42,25.2088,64.6357,57,No hotel found
412,garissa,KE,22.51,-0.4536,39.6401,80,Hiddig Hotel
484,cartagena,CO,25.79,10.3997,-75.5144,83,Hotel Bellamar


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [47]:
# %%capture --no-display

# Configure the map plot
hotel_name_on_map = hotel_df.hvplot.points(
  "Lng",
  "Lat",
  geo = True,
  tiles = "OSM",
  frame_width = 800,
  frame_height = 600,
  size = "Humidity",
  scale = 1,
  color = "City",
)
# Configure the map plot
hotel_name_on_map = hotel_df.hvplot.points(
  "Lng",
  "Lat",
  geo = True,
  tiles = "OSM",
  frame_width = 800,
  frame_height = 600,
  size = "Humidity",
  scale = 1,
  color = "City",
  hover_cols = ["Hotel Name","Country"]
)

# Display the map
hotel_name_on_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)